# iPython notebook to recursively apply permissions to Projects
## Importing libraries

In [ ]:
import os, argparse, keyring, re, configparser, warnings, urllib, requests, webbrowser
from datetime import time, datetime
from getpass import getpass
from pathlib import Path
warnings.filterwarnings('ignore')
config = configparser.ConfigParser()
config.read(r".\project_permissions.cfg")

## Functions
### Fn to check errors

In [ ]:
def check_error(request, task):
    if task == "sign_in":
        if request.status_code == 200:
            print("\t\tUser signed in successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User not found!!")
            return 0
        elif request.status_code == 401:
            print("\t\tERROR: Login error!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "sign_out":
        if request.status_code == 204:
            print("\t\tUser signed out successfully!")
            return 1
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "create_users":
        if request.status_code == 201:
            print("\t\tUser created successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role or bad request!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "update_users":
        if request.status_code == 200:
            print("\t\tUser information updated successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User or Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role, email address or bad request!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Licensing update on self or guest account not allowed!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "find_group_id":
        if request.status_code == 200:
            print("\t\tGroup found!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "add_user_group":
        if request.status_code == 200:
            print("\t\tUser added to group successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site or Group not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: Specified User already in group!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    
    elif task == "query_projects":
        if request.status_code == 200:
            print("\t\tProject found!")
            return 1
        elif request.status_code == 400:
            print("\t\tERROR: Pagination error!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Page not found!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0


### Fn to sign in to Server

In [ ]:
def sign_in(username, password, site=""):
    body = {
        "credentials": {
            "name": username,
            "password": password,
            "site": {
                "contentUrl": site
            }
        }
    }
    response = requests.post(
        URL + '/auth/signin', 
        json=body, 
        verify=False, 
        headers={'Accept': 'application/json'}
    )
    
    status = check_error(response, "sign_in")
    if status:
        return response.json()['credentials']['site']['id'], response.json()['credentials']['token']
    else:
        return 0,0

### Fn to sign out from Server

In [ ]:
def sign_out(site_id, token):
    response = requests.post(
        URL + '/auth/signout', 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )
    status = check_error(response, "sign_out")
    return status

### Fn to find project ids

In [ ]:
def query_projects(site_id, token, project_names):
    projects = []
    if len(project_names) > 0:
        for project in project_names:
            response = requests.get(
                URL + '/sites/{}/projects?filter=name:eq:{}'.format(site_id, urllib.parse.quote_plus(project)), 
                verify=False, 
                headers={'Accept': 'application/json',
                        'X-Tableau-Auth': token}
            )
            status = check_error(response, "query_projects")
            temp = {response.json()['projects']['project'][0]['name']:response.json()['projects']['project'][0]['id']}
            projects.append(temp)
    else:
        response = requests.get(
            URL + '/sites/{}/projects'.format(site_id), 
            verify=False, 
            headers={'Accept': 'application/json',
                    'X-Tableau-Auth': token}
        )
        status = check_error(response, "query_projects")
        page_size = int(response.json()['pagination']['pageSize'])
        items_available = int(response.json()['pagination']['totalAvailable'])
        if items_available%page_size > 0:
            num_pages = int(items_available/page_size) + 1
        else:
            num_pages = int(items_available/page_size)
        for page_num in range(num_pages):
            response = requests.get(
                URL + '/sites/{}/projects?pageSize={}&pageNumber={}'.format(site_id, page_size, int(page_num)+1), 
                headers={'Accept': 'application/json',
                        'X-Tableau-Auth': token}
            ).json()
            project_names = response['projects']['project']
            for project in project_names:
                temp = {project['name']:project['id']}
                projects.append(temp)
    return projects

### Fn to apply Project specific permissions

In [ ]:
def apply_project_permissions(site_id, token, user, capabilities):
     project_permissions_body = {
        "permissions": {
            "granteeCapabilities": {
                "user" : {
                    "id" : user
                }
                "capabilities" : {
                    
                }
            } user_name,
            "siteRole": site_role
        }
    }

## Variables

In [ ]:
server = config["server_connection"]["server"] # Enter site in format tableau.company.com without the https before it
site_content_url = config["server_connection"]["site"] # This can be found from the URL of the content and if using the Default site then this will be blank
api_ver = config["server_connection"]["api"] # This can be found from the Tableau Server REST API reference
username = "pnl0dv6z" # This is your username

URL = "https://{}/api/{}".format(server, api_ver)

## Open Log file

In [ ]:
log_file_loc = r"{}\{}".format(str(Path.home()), config["logging_details"]["logfilename"])
log_file = open(log_file_loc, "a+")

## Sign in to the Tableau Server

In [ ]:
password = getpass("Enter your password for the Tableau Server: ")
site_id, token = sign_in(username, password, site_content_url)
if token == 0:
    exit()

In [ ]:
test = query_projects(site_id, token, [])

In [ ]:
print(len(test))

In [ ]:
response = requests.get(
    URL + '/sites/{}/projects?pageSize=100&pageNumber=5'.format(site_id), 
    headers={'Accept': 'application/json',
            'X-Tableau-Auth': token}
).json()
print(response)

In [ ]:
n = 100
d = 100
print(n%d)

In [ ]:
print(test)

In [ ]:
urllib.parse.quote_plus('analytics_service')

In [ ]:
test_dict = {'write':'allow', 'read':'allow'}

In [ ]:
cap_dict = []
for i in test_dict:
    temp = {}
    temp['capability'] = {'name': i, 'mode' : test_dict[i]}
    cap_dict.append(temp)
print(cap_dict)

In [ ]:
project_permissions_body = {
        "permissions": {
            "granteeCapabilities": {
                "user" : {
                    "id" : 'pnl0dv6z'
                }
                "capability":
                    cap_dict
            }
        }
    }

In [ ]:
type(project_permissions_body)